# Project 3: Web APIs & NLP

### By Alex Lau

## Table of Contents

- [Problem Statement](#Problem-Statement)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
    - [Import Packages and Data](#Import-Packages-and-Data)
    - [High-Level Checks](#High-Level-Checks)
    - [Investigating Target Variable](#Investigating-Target-Variable)
    - [Investigating Features](#Investigating-Features)
    - [Visualizing Top 5 Features by Correlation](#Visualizing-Top-5-Features-by-Correlation)
    - [Handling Outliers](#Handling-Outliers)
- [Data Cleaning](#Data-Cleaning)
    - [Converting Ordinal Features to Numbers](#Converting-Ordinal-Features-to-Numbers)
    - [Inputting Missing Values: Nominal](#Inputting-Missing-Values:-Nominal)
    - [Inputting Missing Values: Continuous](#Inpu`tting-Missing-Values:-Continuous)
    - [Inputting Missing Values: Discrete](#Inputting-Missing-Values:-Discrete)
- [Feature Engineering](#Feature-Engineering)
    - [Dummy Variable Creation](#Dummy-Variable-Creation)
    - [Interaction Variable Creation](#Interaction-Variable-Creation)
    - [Ensuring Consistent Columns for Both Dataframes](#Ensuring-Consistent-Columns-for-Both-Dataframes)
    - [Reviewing Correlations](#Reviewing-Correlations)
- [Preprocessing](#Preprocessing)
- [Modeling](#Modeling)
    - [Logistic Regression](#Logistic-Regression)
    - [Gaussian Naive Bayes](#Gaussian-Naive-Bayes)
- [Conclusions and Evaluation](#Conclusions-and-Evaluation)

## Problem Statement

## Exploratory Data Analysis

### Import Packages and Data

In [3]:
# Import libaries
import pandas as pd
import numpy as np
import requests
import datetime as dt
import time

from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.naive_bayes import GaussianNB, MultinomialNB


In [ ]:
## Save url as string
BASE_URL = 'http://www.mediaeater.com/cameras/'
com_board_1 = 'info/cb-01.html'

url = BASE_URL + com_board_1

## Data Cleaning

## Feature Engineering

## Modeling

### Logistic Regression

### Gaussian Naive Bayes

## Conclusions and Evaluation